In [16]:

import requests
import json
from datetime import datetime, timedelta
import time
import pandas as pd
import os

base_url = "https://api.bitflyer.jp"
endpoint = "/v1/board?product_code="
pair = "BTC_JPY"

data = requests.get(base_url + endpoint + pair, timeout=5)


# スリープ秒数
SLEEP_T = 1

# データ保存の時間[s]
DATA_T = 3600

# boardデータ取得関数

base_url = "https://api.bitflyer.jp"
endpoint = "/v1/board?product_code="

# 対象
pair = "BTC_JPY"


def get_board(designated_time):
    # board = api.board(product_code=source)
    data = requests.get(base_url + endpoint + pair, timeout=5)
    board = json.loads(data.text)
    dict_data = {}
    dict_data['time'] = "{:0>14}".format(designated_time.strftime("%Y%m%d%H%M%S"))
    mid_price = {'mid_price': board['mid_price']}
    dict_data.update(mid_price)
    asks_price = {'asks_price_{}'.format(i): board["asks"][i]["price"] for i in range(10)}
    dict_data.update(asks_price)
    asks_size = {'asks_size_{}'.format(i): board["asks"][i]["size"] for i in range(10)}
    dict_data.update(asks_size)
    bids_price = {'bids_price_{}'.format(i): board["bids"][i]["price"] for i in range(10)}
    dict_data.update(bids_price)
    bids_size = {'bids_size_{}'.format(i): board["bids"][i]["size"] for i in range(10)}
    dict_data.update(bids_size)

    return pd.Series(dict_data)


def get_btc_board(init_time,end_time):
    main_list = []
    designated_time = init_time
    while designated_time < end_time:
        try:
            dict_data = get_board(designated_time)
            main_list.append(dict_data)
            print("dict_data "+dict_data)
        except Exception as e:
            print("exception: ", e.args)
        # sleep
        time.sleep(SLEEP_T)

        # startを1秒進める
        designated_time += timedelta(seconds=1)

    df_data = pd.concat(main_list, axis=1).T
    df_data.to_csv('./data/btc_{}{}{}{}{}{}-{}{}{}{}{}{}.csv'.format("{:0>4}".format(init_time.year),"{:0>2}".format(init_time.month), "{:0>2}".format(init_time.day), "{:0>2}".format(init_time.hour), "{:0>2}".format(init_time.minute),"{:0>2}".format(init_time.second),"{:0>4}".format(end_time.year), "{:0>2}".format(end_time.month), "{:0>2}".format(end_time.day), "{:0>2}".format(end_time.hour), "{:0>2}".format(end_time.minute),"{:0>2}".format((end_time-timedelta(seconds=1)).second)))

if __name__ == '__main__':
    # カレントディレクトリー以下にdataを作成
    if not os.path.exists('./data'):
        os.mkdir('./data')
    init_time = datetime(2023, 4, 17, 0, 0, 0)
    end_time = datetime(2023, 4, 17, 0, 1, 0)
    interval = 1*20

    interim_start_time = init_time
    interim_end_time = interim_start_time+timedelta(seconds=interval)
    
    # ループ処理
    while interim_end_time <= end_time:
        try:
            get_btc_board(interim_start_time,interim_end_time)

        except Exception as e:
            print("exception: ", e.args)

        interim_start_time += timedelta(seconds=interval)
        interim_end_time += timedelta(seconds=interval)



exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to str',)
exception:  ('can only concatenate str (not "float") to

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import datetime
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

%matplotlib inline

# 取得したデータのあるpathに設定
data_path = os.getcwd()+'/data/*.csv'
print("data_path:"+data_path)
data_path_list = glob.glob(data_path)


df_bitcoin_list = []
for i in data_path_list:
    df_bitcoin = pd.read_csv(i, index_col=0) 
    df_bitcoin['time'] = pd.to_datetime(df_bitcoin['time'] )
    df_bitcoin_list.append(df_bitcoin)
df_bitcoin = pd.concat(df_bitcoin_list)

df_bitcoin.index= df_bitcoin.time


# リサンプリングする間隔
interval = '10S'
df_bitcoin_resampling = df_bitcoin.resample(interval).agg([np.max, np.mean, np.sum,np.min, np.std])
df_bitcoin_resampling['count'] = df_bitcoin.resample(interval).count().time

df_bitcoin_resampling_flat = pd.DataFrame(df_bitcoin_resampling.to_records())
df_bitcoin_resampling.columns= [''.join(col_name) for col_name in df_bitcoin_resampling.columns]

y = df_bitcoin.resample(interval).last()['mid_price'].tolist()

x_data = df_bitcoin_resampling.iloc[:-1]
y_data = y[1:] 


#　検証データと学習データ分ける
t_size = int(len(x_data)*0.75)

X_train = x_data[:t_size]
y_train = y_data[:t_size]
X_test = x_data[t_size:]
y_test = y_data[t_size:]

# 計算モデル定義、学習
model = xgb.XGBRegressor()
model.fit(X_train,y_train,
              early_stopping_rounds=10,
              eval_metric = 'mae',
             eval_set=[[X_test, y_test]])

# 予測
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# MAEで評価
print('from sklearn.metrics import mean_absolute_error train : %.3f, test : %.3f' % (mean_absolute_error(y_train, y_train_pred), mean_absolute_error(y_test, y_test_pred)) )

# 可視化
plt.plot(y_test_pred, label="predict")
plt.plot(y_test, label="true")
plt.xlabel("time(/10s)")
plt.ylabel("values(BTC)")
plt.legend()
plt.show()
